<a href="https://colab.research.google.com/github/alenzhasulan/dl-nlp/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22nmt_with_attention_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural machine translation with attention

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import pandas as pd
import unicodedata
import re
import numpy as np
import os
import io
import time
from string import digits

In [2]:
data_path = "mfa_gov_kz_corpus.tsv"

In [3]:
df = pd.read_csv('mfa_gov_kz_corpus.tsv',sep='\t', error_bad_lines=False,names=['source','target', 'coef'])
df.head()

,source,target,coef
0,лисаббон .,lisbon .,1.225
1,сайтта инвестициялауды қажет ететін жобалар ту...,visitors can also send requests through the si...,"0,04"
2,тараптар осы жылдың екінші жартысында еқыұ мен...,"the first president offered a new "" 3d "" geopo...","0,09"
3,"өз кезегінде "" kazakh invest "" ұк "" ақ басқарм...",volume of investments in the region has been g...,"0,13"
4,анықтама :,note to editors .,"0,15"


In [4]:
#Read the data
lines_raw= pd.read_csv('mfa_gov_kz_corpus.tsv',sep='\t', error_bad_lines=False,names=['source','target', 'coef'])
lines_raw.sample(5)

,source,target,coef
6536,"астана , 2018 жылғы 20 қараша астана , 2018 жы...","astana , november 20 , 2018 - kazakhstan-russi...","1,44"
4356,"kazakh invest ұлттық компаниясы "" ақ қр сырт...",kazakh invest is a subordinate organization of...,"1,05"
5988,бұл іс-шараның қазақстан үкіметінің гендерлік...,"1,30",NaN
2994,қ . әбдірахманов халықаралық аренада қазіргі б...,kairat abdrakhmanov emphasized the role of the...,"0,90"
7784,қазақстан 2014 жылдың қараша айынан бастап 1 м...,kazakhstan since november 2014 is an observer ...,"2,29"


In [5]:
kaz= df['source'][:100].str.lower()
en= df['target'][:100].str.lower()


In [6]:
sample_size=60000
source='start_ ' + kaz + ' _end'
target ='start_ ' + en + ' _end'
source

0                                start_ лисаббон . _end
1     start_ сайтта инвестициялауды қажет ететін жоб...
2     start_ тараптар осы жылдың екінші жартысында е...
3     start_ өз кезегінде " kazakh invest " ұк " ақ ...
4                                start_ анықтама : _end
                            ...                        
95    start_ қазіргі таңда қазақстандық бидайды мысы...
96    start_ конференция барысында халықаралық терро...
97    start_ аустрияның " öwd wachdienst security gm...
98    start_ мысалы , жапон орта мектептерінде дрк т...
99    start_ қ . абдрахманов " төмен байытылған уран...
Name: source, Length: 100, dtype: object

In [7]:
def max_length(tensor):
  return max(len(t) for t in tensor)

__Create the source and ttarget toekns and post pad them__

In [8]:
source_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
source_sentence_tokenizer.fit_on_texts(source)
source_tensor = source_sentence_tokenizer.texts_to_sequences(source)
source_tensor= tf.keras.preprocessing.sequence.pad_sequences(source_tensor,padding='post' )

In [10]:
target_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
target_sentence_tokenizer.fit_on_texts(target)
target_tensor = target_sentence_tokenizer.texts_to_sequences(target)
target_tensor= tf.keras.preprocessing.sequence.pad_sequences(target_tensor,padding='post' )
print(len(target_tensor[0]))

58


In [11]:
max_target_length= max(len(t) for t in  target_tensor)
print(max_target_length)
max_source_length= max(len(t) for t in  source_tensor)
print(max_source_length)

58
79


##  Creating Train and Test dataset

In [12]:
source_train_tensor, source_test_tensor, target_train_tensor, target_test_tensor= train_test_split(source_tensor, target_tensor,test_size=0.2)

In [13]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(source_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80 80 20 20


In [14]:
type(input_tensor_train)

numpy.ndarray

In [15]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [16]:
print ("Input Language; index to word mapping")
convert(source_sentence_tokenizer, source_train_tensor[0])
print ()
print ("Target Language; index to word mapping")
convert( target_sentence_tokenizer, target_train_tensor[0])

Input Language; index to word mapping
1 ----> start_
878 ----> талқылаудың
94 ----> маңызды
879 ----> тақырыптарының
96 ----> бірі
880 ----> нұр-сұлтанда
38 ----> еқыұ
881 ----> тақырыптық
882 ----> орталығын
31 ----> құру
9 ----> туралы
13 ----> қазақстандық
883 ----> бастама
203 ----> болды
3 ----> .
2 ----> _end

Target Language; index to word mapping
3 ----> start_
26 ----> one
2 ----> of
1 ----> the
58 ----> main
114 ----> topics
2 ----> of
218 ----> discussion
12 ----> was
1 ----> the
14 ----> kazakh
612 ----> initiative
9 ----> to
613 ----> establish
45 ----> an
37 ----> osce
614 ----> thematic
71 ----> center
7 ----> in
98 ----> nur-sultan
15 ----> on
615 ----> sustainable
616 ----> interconnection
6 ----> ,
617 ----> focused
15 ----> on
618 ----> analytic
8 ----> and
619 ----> research
219 ----> work
5 ----> .
4 ----> _end


### Create a tf.data dataset

In [17]:
BUFFER_SIZE = len(source_train_tensor)
BATCH_SIZE = 64
steps_per_epoch = len(source_train_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(source_sentence_tokenizer.word_index)+1
vocab_tar_size = len(target_sentence_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((source_train_tensor, target_train_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
type(dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [18]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 79]), TensorShape([64, 58]))

In [19]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [20]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 79, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [21]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [22]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 79, 1)


In [23]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [24]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 854)


## Define the optimizer and the loss function

In [25]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [26]:
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [27]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [28]:
steps_per_epoch

1

In [30]:
EPOCHS = 200

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
    if batch % 100 == 0:
      print('Epoch {} Batch {} loss {}'.format(epoch + 1,batch, batch_loss.numpy()))
   
      
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 loss 2.5616629123687744
Epoch 1 Loss 2.5617
Time taken for 1 epoch 0.8157966136932373 sec

Epoch 2 Batch 0 loss 2.506190061569214
Epoch 2 Loss 2.5062
Time taken for 1 epoch 1.0423014163970947 sec

Epoch 3 Batch 0 loss 2.5563480854034424
Epoch 3 Loss 2.5563
Time taken for 1 epoch 0.7663540840148926 sec

Epoch 4 Batch 0 loss 2.4221394062042236
Epoch 4 Loss 2.4221
Time taken for 1 epoch 1.0684523582458496 sec

Epoch 5 Batch 0 loss 2.3820948600769043
Epoch 5 Loss 2.3821
Time taken for 1 epoch 0.7500848770141602 sec

Epoch 6 Batch 0 loss 2.375213384628296
Epoch 6 Loss 2.3752
Time taken for 1 epoch 1.0652902126312256 sec

Epoch 7 Batch 0 loss 2.320357322692871
Epoch 7 Loss 2.3204
Time taken for 1 epoch 0.7514598369598389 sec

Epoch 8 Batch 0 loss 2.2855160236358643
Epoch 8 Loss 2.2855
Time taken for 1 epoch 1.0525424480438232 sec

Epoch 9 Batch 0 loss 2.2555060386657715
Epoch 9 Loss 2.2555
Time taken for 1 epoch 0.7754132747650146 sec

Epoch 10 Batch 0 loss 2.0517735481262207

In [36]:
def evaluate(sentence):
  attention_plot = np.zeros((max_target_length, max_source_length))

  sentence = 'start_ ' + sentence + ' _end'
  #print(sentence)
  #print(source_sentence_tokenizer.word_index)

  inputs = [source_sentence_tokenizer.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_source_length,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']], 0)

  for t in range(max_target_length):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += target_sentence_tokenizer.index_word[predicted_id] + ' '

    if target_sentence_tokenizer.index_word[predicted_id] == '_end':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

## To Plot Attention weights

In [37]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [38]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))
  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## Restore the latest checkpoint and test

In [39]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Final translations with Attention Plots

In [44]:
translate('американдық авторлар арасында')

Input: start_ американдық авторлар арасында _end
Predicted translation: outcome _end 
